In [1]:
import json
import pandas as pd
from v2_rebalance_dashboard.constants import ROOT_DIR, eth_client
import plotly.express as px
from v2_rebalance_dashboard.get_rebalance_events_summary import fetch_clean_rebalance_events
import plotly.graph_objects as go

autoETH = "0x49C4719EaCc746b87703F964F09C22751F397BA0"
balETH = "0x72cf6d7C85FfD73F18a83989E7BA8C1c30211b73"


def load_solver_df() -> pd.DataFrame:
    fetched_data_path = ROOT_DIR.parent / "fetched_data"
    existing_jsons = [str(path) for path in fetched_data_path.glob("*.json")]

    all_data = []
    for p in existing_jsons:
        try:
            with open(p, "r") as fin:
                json_file_file_name = p.split("/")[-1]
                solver_data = json.load(fin)
                solver_data["json_file_file_name"] = json_file_file_name
                solver_data["date"] = pd.to_datetime(solver_data["timestamp"], unit="s")

                if autoETH.lower() in json_file_file_name.lower():
                    solver_data["poolAddress"] = autoETH

                if balETH.lower() in json_file_file_name.lower():
                    solver_data["poolAddress"] = balETH

                all_data.append(solver_data)
        except Exception as e:
            pass
            # print(p, e)
    solver_df = pd.DataFrame.from_records(all_data)
    return solver_df


def load_balETH_solver_df():
    destination_df = pd.read_parquet(ROOT_DIR / "vaults.parquet")
    rebalance_event_df = fetch_clean_rebalance_events()

    destination_vault_to_name = {
        str(vault_address).lower(): name[22:]
        for vault_address, name in zip(destination_df["vaultAddress"], destination_df["name"])
    }
    destination_vault_to_name["0x72cf6d7c85ffd73f18a83989e7ba8c1c30211b73"] = "balETH idle"
    solver_df = load_solver_df()
    balETH_solver_df = solver_df[solver_df["poolAddress"] == balETH].copy()
    balETH_solver_df.set_index("date", inplace=True)

    balETH_solver_df["destinationInName"] = balETH_solver_df.apply(
        lambda row: (
            destination_vault_to_name[row["destinationIn"].lower()]
            if row["destinationIn"].lower() in destination_vault_to_name
            else None
        ),
        axis=1,
    )
    balETH_solver_df["destinationOutName"] = balETH_solver_df.apply(
        lambda row: (
            destination_vault_to_name[row["destinationOut"].lower()]
            if row["destinationOut"].lower() in destination_vault_to_name
            else None
        ),
        axis=1,
    )

    balETH_solver_df["moveName"] = balETH_solver_df.apply(
        lambda row: f"Exit {row['destinationOutName']} enter {row['destinationInName']}", axis=1
    )
    rebalance_event_df["moveName"] = rebalance_event_df.apply(
        lambda row: f"Exit {row['out_destination']} enter {row['in_destination']}", axis=1
    )
    return balETH_solver_df, destination_df, rebalance_event_df


balETH_solver_df, destination_df, rebalance_event_df = load_balETH_solver_df()

In [14]:
def make_proposed_vs_actual_rebalance_scatter_plot(
    balETH_solver_df: pd.DataFrame, rebalance_event_df: pd.DataFrame
) -> go.Figure:
    moves_df = pd.concat([balETH_solver_df["moveName"], rebalance_event_df["moveName"]], axis=1)
    moves_df.columns = ["proposed_rebalances", "actual_rebalances"]
    proposed_rebalances_fig = go.Scatter(
        x=moves_df.index,
        y=moves_df["proposed_rebalances"],
        mode="markers",
        name="Proposed Rebalances",
        marker=dict(color="blue", size=10),
    )

    # Create the plot with actual rebalances with red 'x' markers
    actual_rebalances_fig = go.Scatter(
        x=moves_df.index,
        y=moves_df["actual_rebalances"],
        mode="markers",
        name="Actual Rebalances",
        marker=dict(symbol="x", color="red", size=12),
    )

    # Combine both plots into one figure
    proposed_vs_actual_rebalance_scatter_plot_fig = go.Figure(data=[proposed_rebalances_fig, actual_rebalances_fig])

    # Update layout
    proposed_vs_actual_rebalance_scatter_plot_fig.update_layout(
        yaxis_title="Rebalances",
        xaxis_title="Date",
        title="balETH Proposed vs Actual Rebalances",
        height=600,
        width=600 * 3,
    )
    return proposed_vs_actual_rebalance_scatter_plot_fig


proposed_vs_actual_rebalance_scatter_plot_fig = make_proposed_vs_actual_rebalance_scatter_plot(
    balETH_solver_df, rebalance_event_df
)
proposed_vs_actual_rebalance_scatter_plot_fig